# Data Exploration

In this notebook describe your data exploration steps.

## Install dependencies

In [8]:
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm.auto import tqdm
tqdm.pandas()

## Load data

In [9]:
LSA = pd.read_csv('LSA.csv')
bussgeld = pd.read_csv('bussgeld.csv')

### Look at the first rows

In [10]:
LSA.head()

,LSA-Nr.,Standort,X,Y
0,1617,Aachener Straße / Aachener Weiher / Richard-Wa...,6.928991127434393,50.93643440813894
1,3634,Aachener Straße / Alter Militärring / Auf dem ...,6.881019581929463,50.93772515159249
2,3656,Aachener Straße / Am Rapohl / Bodelschwinghstraße,6.822827155149226,50.93851403303879
3,3639,Aachener Straße / Am Römerhof,6.867657727348872,50.93780279398538
4,3638,Aachener Straße / Am Sportpark Müngersdorf / M...,6.870816665405981,50.93783319888283


In [11]:
indexL = []
print(LSA.shape)
for index, row in LSA.iterrows():
    # Check if the column value contains a comma
    if ',' in str(row['X']):
        # Remove the row
        indexL.append(index)
len(indexL)
print(LSA.shape)
LSA.drop(indexL, inplace=True)
print(LSA.shape)

(1969, 4)
(1969, 4)
(1857, 4)


In [12]:
geolocator = Nominatim(user_agent="sakiExploration")
def get_zipcode(row):
    zipcode = 0
    location = geolocator.reverse((row['Y'], row['X']))
    try:
        zipcode = location.raw['address']['postcode']
        return zipcode
    except:
        return zipcode

In [13]:
LSA['zipcode'] = LSA.progress_apply(get_zipcode, axis = 1)

  0%|          | 0/1857 [00:00<?, ?it/s]

In [14]:
LSA = LSA[LSA['zipcode'] != 0]
LSA.head()

,LSA-Nr.,Standort,X,Y,zipcode
0,1617,Aachener Straße / Aachener Weiher / Richard-Wa...,6.928991127434393,50.93643440813894,50674
1,3634,Aachener Straße / Alter Militärring / Auf dem ...,6.881019581929463,50.93772515159249,50933
2,3656,Aachener Straße / Am Rapohl / Bodelschwinghstraße,6.822827155149226,50.93851403303879,50859
3,3639,Aachener Straße / Am Römerhof,6.867657727348872,50.93780279398538,50933
4,3638,Aachener Straße / Am Sportpark Müngersdorf / M...,6.870816665405981,50.93783319888283,50933


In [23]:
LSA['X'] = LSA['X'].astype(float)
LSA['Y'] = LSA['Y'].astype(float)

In [24]:
LSA.to_csv('LSA.csv', index=False)  

In [16]:
bussgeld = bussgeld[['datum_von', 'kennzeichen1', 'fahrzeugart', 'fabrikat', 'latAndlong']]
bussgeld['latAndlong'] = bussgeld['latAndlong'].progress_apply(lambda x : eval(x))
bussgeld[['Y','X']] = bussgeld['latAndlong'].progress_apply(pd.Series)
bussgeld = bussgeld[bussgeld['X'] != 0.0]
bussgeld.drop('latAndlong', axis=1)
bussgeld.head()


  0%|          | 0/147018 [00:00<?, ?it/s]

  0%|          | 0/147018 [00:00<?, ?it/s]

,datum_von,kennzeichen1,fahrzeugart,fabrikat,latAndlong,Y,X
0,2021-01-14 08:32:00.000,MG,PKW,FIAT,"(50.9316361, 6.9370651)",50.931636,6.937065
1,2021-01-14 13:23:00.000,BM,PKW,OPEL,"(50.9343146, 6.9391205)",50.934315,6.939120
2,2021-01-14 13:23:00.000,K,PKW,VW-VOLKSWAGEN,"(50.9340377, 6.9392313122363465)",50.934038,6.939231
3,2021-01-14 09:25:00.000,K,PKW,FORD,"(50.9336412, 6.9353042)",50.933641,6.935304
4,2021-01-14 14:25:00.000,M,PKW,VW-VOLKSWAGEN,"(50.9363573, 6.9327157)",50.936357,6.932716


In [60]:
from scipy.spatial import distance
def get_nearest_trafficlight(X, Y, dataTrafficLight):
    # Define the violation location coordinates (latitude and longitude)
    violation_X= X 
    violation_Y = Y 
    
    # Calculate the distance between the violation location and each traffic light
    distances = []
    for index, row in dataTrafficLight.iterrows():
        light_X = row['X']  
        light_Y = row['Y']  

        # Calculate the Euclidean distance between the violation location and the current traffic light
        dist = distance.euclidean((violation_X, violation_Y), (light_X, light_Y))
        distances.append(dist)

    # Find the index of the nearest traffic light
    nearest_light_index = distances.index(min(distances))
    return dataTrafficLight.iloc[nearest_light_index]['LSA-Nr.']

In [79]:
bussgeld['LSA_Nr'] = bussgeld[['X','Y']].progress_apply(lambda r: get_nearest_trafficlight(r['X'],r['Y'],LSA), axis=1)

  0%|          | 0/142491 [00:00<?, ?it/s]

In [84]:
LSA.rename(columns={"LSA-Nr.": "LSA_Nr", 'X': "LSA_X", "Y": "LSA_Y"},inplace= True)

In [88]:
LSA.rename(columns={'X': "LSA_X", "Y": "LSA_Y"},inplace= True)

In [89]:
LSA

,LSA_Nr,Standort,LSA_X,LSA_Y,zipcode
0,1617,Aachener Straße / Aachener Weiher / Richard-Wa...,6.928991,50.936434,50674
1,3634,Aachener Straße / Alter Militärring / Auf dem ...,6.881020,50.937725,50933
2,3656,Aachener Straße / Am Rapohl / Bodelschwinghstraße,6.822827,50.938514,50859
3,3639,Aachener Straße / Am Römerhof,6.867658,50.937803,50933
4,3638,Aachener Straße / Am Sportpark Müngersdorf / M...,6.870817,50.937833,50933
...,...,...,...,...,...
1964,99966,Schlagbaumsweg / Colonia Allee / Isenburger Ki...,7.044873,50.959772,51067
1965,99967,Bergisch Gladbacher Str. / Steyler Str.,7.035198,50.967521,51067
1966,99968,Mielenforster Str. / Bensberger Marktweg,7.073978,50.967476,51069
1967,99969,Heidelberger Str. / Kopernikusstr.,7.010068,50.952261,51065


In [90]:
merged_data = pd.merge(bussgeld, LSA, on="LSA_Nr")

In [92]:
merged_data.to_csv("finalData.csv")